In [1]:
# Import necessary libraries
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.layers import Layer
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
import pickle

# Import custom modules
from network import *
from data import *

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dbfile = open('../../Data/full', 'rb')    
data = pickle.load(dbfile)
dbfile = open('../../Data/location_pert', 'rb')    
location_pert = pickle.load(dbfile)
dbfile = open('../../Data/pert2profiles', 'rb')    
pert2profiles = pickle.load(dbfile)
dbfile = open('../../Data/test_perts', 'rb')    
test_pert = pickle.load(dbfile)
dbfile = open('../../Data/train_perts', 'rb')    
train_pert = pickle.load(dbfile)
dbfile = open('../../Data/X_test_triplet', 'rb')
X_test = pickle.load(dbfile)
dbfile = open('../../Data/X_train_triplet', 'rb')
X_train = pickle.load(dbfile)
dbfile.close()

# Network

In [3]:
# Create the base network
network = dict(n_dense=5, dense_units=16, activation='selu', dropout=AlphaDropout, dropout_rate=0.1,
               kernel_initializer='lecun_normal', optimizer='sgd', num_classes=2)

shared_model = create_base_network(**network)

In [4]:
# Create the siamese network
model = create_siamese_network(shared_model)
print("Siamese network model summary")
model.summary()

y_pred.shape =  Tensor("merged_layer/concat:0", shape=(?, 48), dtype=float32)
Siamese network model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 978)          0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 978)          0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 978)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 16)           16752       anchor_input[0][0]               
 

In [5]:
y_dum = np.zeros((len(X_train[0]),1))
model.fit([X_train[0],X_train[1],X_train[2]],y_dum,epochs=100)

Epoch 1/100
420/420 [==============================] - 1s 3ms/step - loss: 6.4835
Epoch 2/100
420/420 [==============================] - 0s 196us/step - loss: 4.5725
Epoch 3/100
420/420 [==============================] - 0s 186us/step - loss: 4.6885
Epoch 4/100
420/420 [==============================] - 0s 184us/step - loss: 3.8149
Epoch 5/100
420/420 [==============================] - 0s 187us/step - loss: 4.0711
Epoch 6/100
420/420 [==============================] - 0s 183us/step - loss: 2.8982
Epoch 7/100
420/420 [==============================] - 0s 183us/step - loss: 2.9066
Epoch 8/100
420/420 [==============================] - 0s 189us/step - loss: 2.7519
Epoch 9/100
420/420 [==============================] - 0s 191us/step - loss: 2.8610
Epoch 10/100
420/420 [==============================] - 0s 194us/step - loss: 2.4655
Epoch 11/100
420/420 [==============================] - 0s 185us/step - loss: 2.5663
Epoch 12/100
420/420 [==============================] - 0s 197us/step - loss

420/420 [==============================] - 0s 207us/step - loss: 0.1169
Epoch 98/100
420/420 [==============================] - 0s 168us/step - loss: 0.0581
Epoch 99/100
420/420 [==============================] - 0s 181us/step - loss: 0.0668
Epoch 100/100
420/420 [==============================] - 0s 167us/step - loss: 0.1023


In [6]:
pred1 = model.predict([X_test[0],X_test[1],X_test[2]])
pred1.shape

(420, 48)

In [7]:
pred = model.predict([X_train[0],X_train[1],X_train[2]])
pred.shape

(420, 48)

In [8]:
from sklearn.preprocessing import normalize
anch = normalize(pred[0][0:16].reshape(1,-1)).flatten()
pos = normalize(pred[0][16:32].reshape(1,-1)).flatten()
neg = normalize(pred[0][32:].reshape(1,-1)).flatten()

In [9]:
def softmax_function(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [36]:
values = []
for j in np.arange(0,1,0.01):
    for i in pred1:
        anch = normalize(i[0:16].reshape(1,-1)).flatten()
        pos = normalize(i[16:32].reshape(1,-1)).flatten()
        neg = normalize(i[32:].reshape(1,-1)).flatten()
        pos_neg_val = [(np.dot(anch,pos)), (np.dot(anch,neg))]
        #print(pos_neg_val)
        values.append(pos_neg_val[0]>=j)
        values.append(pos_neg_val[1]<j)
    # print(positives, negatives)
    #     print(pos_val, neg_val, pos_val>neg_val)
    print((np.sum(np.asarray(values)==True))/len(values))

0.5773809523809523
0.5779761904761904
0.5785714285714286
0.5785714285714286
0.5785714285714286
0.5787698412698413
0.5787414965986395
0.5785714285714286
0.5784391534391534
0.5783333333333334
0.5783549783549784
0.5783730158730159
0.5782051282051283
0.5780612244897959
0.5777777777777777
0.5775297619047619
0.5773109243697478
0.5770502645502645
0.5768796992481203
0.5766666666666667
0.5765873015873015
0.5764069264069264
0.5762939958592133
0.5764384920634921
0.5763809523809523
0.5763736263736263
0.5763227513227513
0.5762329931972789
0.5761494252873564
0.5760714285714286
0.5760752688172043
0.5760788690476191
0.576010101010101
0.5759453781512605
0.5758503401360544
0.575760582010582
0.5756756756756757
0.5756265664160402
0.5756105006105006
0.575625
0.5756097560975609
0.575595238095238
0.575609080841639
0.5756222943722944
0.5757142857142857
0.5758022774327122
0.5758865248226951
0.5759176587301588
0.5759718172983479
0.5759761904761905
0.5759803921568627
0.5759844322344322
0.576010781671159
0.576036

In [16]:
pos_neg_val

array([0.51011103, 0.4898889 ], dtype=float32)

In [22]:
pred1.shape

(420, 48)

In [24]:
for i in pred1:
    print(i[0:16],i[16:32],i[32:])

[-1.2936662 -1.4493936 -1.4305114 -1.4165641 -1.049892  -1.3672498
 -1.1021305 -1.5060433 -1.1227076 -1.3851438 -1.1467873 -1.4402779
 -1.4594862 -1.1996304 -1.507519  -1.3197008] [-1.2399225  -1.3976992  -1.4851322  -1.376986   -0.99357796 -1.320479
 -0.9425099  -1.5203084  -1.1049392  -1.4355218  -1.0839634  -1.4825414
 -1.3959268  -1.2398381  -1.5235267  -1.3965758 ] [-1.3562518 -1.3923167 -1.340264  -1.5331918 -1.1508542 -1.2033927
 -1.4227543 -1.2605051 -1.2157532 -1.4731809 -1.0958675 -1.2215365
 -1.5611773 -1.3084526 -1.4156064 -1.2479038]
[-1.45367   -1.4328606 -1.4844368 -1.4844265 -1.1481405 -1.3680946
 -1.0731663 -1.5450084 -1.1138161 -1.3902521 -1.2509725 -1.3480664
 -1.447008  -1.1949656 -1.5200368 -1.2223718] [ 0.51043046  0.45884344 -1.3155316  -0.5592305  -0.49091175  0.50556153
  0.09728821 -1.2191957  -1.0795532  -0.89472765  1.0263072  -0.9870959
 -1.0515276  -1.6043086   0.5159645  -1.6778622 ] [-1.4582381  -0.49861464 -0.8814546  -1.1663232  -1.3874995   0.37517732

  0.9885436  0.7188863  0.2543815  3.0879188] [-1.4015512  -1.3743209   0.5122798  -0.57689613 -1.4996234  -1.3753965
 -1.252696   -1.2547923  -0.31177333 -0.6214452  -1.459446    0.36282632
 -1.1922073  -0.9697483  -1.506717    0.43477583]
[ 3.7536147  5.052665   1.5905474  2.850627   1.9190433  4.164809
  1.9998901  3.9557228  1.6477079  2.908168   3.8760204  2.8795843
  2.4014459 -0.27762    6.4018974 -0.8205746] [-1.4496974  -1.4953966  -0.4054461  -1.1779517  -1.2875229  -1.1900928
 -1.2236022  -1.3911582  -1.0878278  -0.91086924 -1.2878106  -0.6411603
 -1.0774467  -0.7949277  -1.4250295  -0.59730184] [-0.28386006  0.52819246 -1.2580633   0.5867725  -0.4440322   0.06384049
  0.02832194 -0.6064416  -0.31557104 -0.90904385  1.0944384   0.4237849
  0.222836   -1.4969538   1.0692871  -1.2347746 ]
[-1.2532749   0.02599414  1.5339284  -0.51566404 -0.697862   -0.0722354
 -1.2401786   1.1574305   0.5267      0.88876706 -1.2648369   1.6603273
  0.37523028  1.0842801  -0.5255881   2.3379257

 -1.5458261 -1.1407286 -1.4930282 -1.2229531]
[11.968066   8.324622   9.341145  14.660755   5.55811    5.3146377
  9.665178  15.211702   9.134303  10.525218   4.980637   7.2215633
 13.022696   6.0637803 13.602084   8.049823 ] [10.428265   7.7193937  9.930817  13.862201   4.7490053  5.7407837
  8.020273  14.55128    7.8324847 10.42824    5.085809   8.113316
 11.944121   5.3806195 12.752851   8.067177 ] [-1.3702414  -1.2706206   0.09767915 -0.8511353  -1.4878709  -1.4059242
 -1.2384688  -1.2979321  -0.06041886 -0.74717766 -1.4711984   0.08831361
 -1.3196378  -0.9840977  -1.5012375   0.19590363]
[7.5417614 5.1762815 4.737184  6.9492893 3.9095218 3.7222404 5.124994
 9.246754  4.7021146 5.8454556 3.4063523 3.654126  6.7424073 2.8515713
 8.843271  2.3213944] [ 9.763595   8.608621   6.989045  10.1278105  5.091862   4.966232
  6.856043  12.420079   7.333707   8.708878   4.410719   6.1906643
  9.38548    4.5431666 12.103084   5.8814983] [-1.3613745 -1.4063699 -1.3908502 -1.4903034 -1.1096009 -1